In [ ]:
import os  # for file paths
import pandas as pd
import awswrangler as wr
import pydbtools as pydb  # see https://github.com/moj-analytical-services/pydbtools

# few things for viewing dataframes better
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

In [ ]:
# define some variables to be used throughout the notebook
db1 = "familyman_dev_v3" #database where Familyman data is stored
db2 = "fcsq" #database where tables created as part of FCSQ processing are stored where required
#snapshot dates and publication period are set in the main run file. However, if running this notebook independently you will need to set them here
snapshot_date = "2023-01-05"
#pub_year = 2011
#pub_qtr = 4

In [ ]:
# create path for within athena FCSQ database in the S3 folder
fcsq_db_path = f"s3://alpha-family-data/fcsq_processing/CA_apps/"

### Orders Applied for (aggregated)

In [ ]:
#Aggregate up orders applied for
pydb.create_temp_table( 
f"""
SELECT    
  year,
  quarter,
  order_case_type,
  order_code,
  order_desc,
  count(*) AS count
FROM 
  {db2}.ca_apps_order_types
GROUP BY 
  year,
  quarter,
  order_case_type,
  order_code,
  order_desc
""",

"ca_orders_applied")

### Extract relationshipt to child info from Familyman

In [ ]:
#Extract applicant and respondent relationship to child information
pydb.create_temp_table( 
f"""
SELECT    
  value AS rtc,
  field_model,
  role
FROM 
  {db1}.role_fields
WHERE
  field_model IN ('APLC_RC','RSPC_RC')
  AND mojap_snapshot_date = DATE'{snapshot_date}'
""",

"app_resp_rtc")

### RTC - applicants

In [ ]:
#Get rtc for applicants
#As joining applicants using role id it will automatically filter out and rspc_rc records from the app_resp_rtc table
pydb.create_temp_table( 
f"""
SELECT    
  a.case_number,
  a.year,
  a.quarter,
  a.order_case_type,
  a.order_code,
  a.order_desc,
  ap.role_id,
  r.field_model AS app_resp,
  r.rtc
FROM 
  {db2}.ca_apps_order_types a
  LEFT JOIN {db2}.ca_applicants ap 
    ON a.case_number = ap.case_number
  LEFT JOIN __temp__.app_resp_rtc r
    ON ap.role_id = r.role
""",

"apps_rtc")

In [ ]:
#Aggregate up applicant rtc data
pydb.create_temp_table( 
f"""
SELECT    
  year,
  quarter,
  order_case_type,
  order_code,
  order_desc,
  rtc,
  count(*) AS count
FROM 
  __temp__.apps_rtc
GROUP BY
  year,
  quarter,
  order_case_type,
  order_code,
  order_desc,
  rtc
""",

"agg_apps_rtc")

### RTC - Respondents

In [ ]:
#Get rtc for resps
#As joining respondents using role id it will automatically filter out and aplc_rc records from the app_resp_rtc table
pydb.create_temp_table( 
f"""
SELECT    
  a.case_number,
  a.year,
  a.quarter,
  a.order_case_type,
  a.order_code,
  a.order_desc,
  rp.role_id,
  r.field_model AS app_resp,
  r.rtc
FROM 
  {db2}.ca_apps_order_types a
  LEFT JOIN {db2}.ca_respondents rp 
    ON a.case_number = rp.case_number
  LEFT JOIN __temp__.app_resp_rtc r
    ON rp.role_id = r.role
""",

"resps_rtc")

In [ ]:
#Aggregate up respondent rtc data
pydb.create_temp_table( 
f"""
SELECT    
  year,
  quarter,
  order_case_type,
  order_code,
  order_desc,
  rtc,
  count(*) AS count
FROM 
  __temp__.apps_rtc
GROUP BY
  year,
  quarter,
  order_case_type,
  order_code,
  order_desc,
  rtc
""",

"agg_resps_rtc")